In [31]:
import networkx as nx
import ipywidgets as widgets
from ipywidgets import interact
from pyvis.network import Network
from IPython.display import display, HTML
import os


In [32]:

%matplotlib inline


In [33]:
# Define the ordered list of scale degree labels.
scale_degrees = [
    'A2', 'm2', 'P8', 'A6',
    'm7', 'M2', 'm6', 'M7',
    'm3', 'M3', 'P5', 'd7',
    'P4', 'M6', 'A4', 'd5'
]
n_scale = len(scale_degrees)

rest_val = 16

In [34]:

def parse_graph_from_lines(lines):
    """
    Parse a single graph from a list of nonempty lines.
    
    Expected format:
       N=50
       X:
       <node values spanning one or more lines>
       E:
       <N*N edge values spanning one or more lines>
    """
    N = int(lines[0].split("=")[1])
    try:
        x_index = lines.index("X:") + 1
    except ValueError:
        raise ValueError("Could not find 'X:' section in the graph chunk.")
    try:
        e_index = lines.index("E:")
    except ValueError:
        raise ValueError("Could not find 'E:' section in the graph chunk.")
    
    x_values = []
    for line in lines[x_index:e_index]:
        x_values.extend(line.split())
    if len(x_values) != N:
        raise ValueError(f"Expected {N} node values but found {len(x_values)}")
    nodes = [int(val) for val in x_values]
    
    edge_numbers = []
    for line in lines[e_index+1:]:
        edge_numbers.extend(line.split())
    if len(edge_numbers) != N * N:
        raise ValueError(f"Expected {N*N} edge values but found {len(edge_numbers)}")
    
    matrix = []
    for i in range(N):
        row = [int(edge_numbers[i * N + j]) for j in range(N)]
        matrix.append(row)

    upper_triangular = [
        [matrix[i][j] if i <= j else 0 for j in range(len(matrix[0]))]
        for i in range(len(matrix))
    ]
    
    return N, nodes, upper_triangular

def parse_graphs_file(filename):
    """
    Parse a file that contains one or more graphs.
    Each graph is assumed to start with a line beginning with "N=".
    Returns a list of (N, nodes, matrix) tuples.
    """
    with open(filename, "r") as f:
        lines = [line.strip() for line in f if line.strip()]
    
    graphs_data = []
    i = 0
    while i < len(lines):
        if lines[i].startswith("N="):
            current_chunk = []
            # Collect lines until we hit the next "N=" (except for the first line)
            while i < len(lines) and (not (lines[i].startswith("N=") and current_chunk)):
                current_chunk.append(lines[i])
                i += 1
            graphs_data.append(parse_graph_from_lines(current_chunk))
        else:
            i += 1
    return graphs_data

def build_graph(N, node_vals, matrix):
    """
    Build a NetworkX directed graph from node values and the edge matrix.
    Node values of 51 are treated as a rest.
    """
    G = nx.DiGraph()
    for i, val in enumerate(node_vals):
        label = "Rest" if val == rest_val else scale_degrees[val % n_scale]
        G.add_node(i, label=label)
    for i in range(N):
        for j in range(N):
            w = matrix[i][j]
            if w != 0:
                G.add_edge(i, j, weight=w)
    return G


In [35]:

# Parse all graphs from the file.
filename = "generated_samples1.txt"
graphs_data = parse_graphs_file(filename)

# Create widgets for interactive selection.
graph_dropdown = widgets.Dropdown(
    options=[(f"Graph {i}", i) for i in range(len(graphs_data))],
    description="Graph:"
)
threshold_slider = widgets.IntSlider(
    value=0, min=0, max=30, step=1, description="Min Edge Weight:"
)

def update_pyvis(graph_index, threshold):
    """
    Update the interactive PyVis visualization.
    Filters out edges with weight below the given threshold.
    """
    N, node_vals, matrix = graphs_data[graph_index]
    G = build_graph(N, node_vals, matrix)
    
    # Create a filtered copy of the graph.
    H = nx.DiGraph()
    for node, data in G.nodes(data=True):
        H.add_node(node, **data)
    for u, v, data in G.edges(data=True):
        if data['weight'] >= threshold:
            H.add_edge(u, v, **data)
    
    # Create a PyVis network.
    net = Network(height="600px", width="100%", directed=True, notebook=True, cdn_resources='in_line')

    net.from_nx(H)
    
    # Save to a temporary HTML file and display inline.
    temp_html = "temp_graph.html"
    net.show(temp_html)
    with open(temp_html, "r", encoding="utf-8") as f:
        html_content = f.read()
    # display(HTML(html_content))
 




In [36]:
# Create interactive controls.
interact(update_pyvis, graph_index=graph_dropdown, threshold=threshold_slider)

interactive(children=(Dropdown(description='Graph:', options=(('Graph 0', 0), ('Graph 1', 1), ('Graph 2', 2)),…

<function __main__.update_pyvis(graph_index, threshold)>